FERC <> EIA Granular Connections

Notes on the type of problem we are trying to solve:
- A classification problem
- A deterministic problem
- A record linkage problem

Right now, we are using the recordlinkage package. We're using logistic regression classifier because it fits all of the above.

What we still need:
- determine how to restrict the results to one eia record per ferc record!
- Remove false granularities in the MUL
- fine tune the comparison metric for heat rate and total fuel cost.
- add in additional FERC tabels
- more test data!
- so, so much more.

To consider:
- Maybe we want to run the records with fuel cost data through a different matching model...

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import pudl.extract.ferc1
import sqlalchemy as sa
import logging
import sys
import copy
from copy import deepcopy

import yaml

from pudl.output.ferc1 import *

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [5]:
sys.path.append("../")
from plant_part_agg_eia import *
from plant_parts import plant_parts
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [6]:
def cleanstrings_snake(df, cols):
    for col in cols:
        df.loc[:, col] = (
            df[col].astype(str).
            str.strip().
            str.lower().
            str.replace(r'\s+', '_')
        )
    return df

### EIA Parts

In [7]:
compiler = CompileTables(pudl_engine, freq='AS', rolling=True)

In [8]:
compiled_plant_parts, plant_parts_df = compiler.generate_master_unit_list(plant_parts)

beginning the aggregation for generators_eia860
grabbing generators_eia860 from the sqlite db
Converting the dtypes of: generators_eia860
aggregate the parts
beginning the aggregation for generation_eia923
grabbing generation_eia923 from the sqlite db
aggregate the parts
Converting the dtypes of: generation_eia923
aggregate the parts
beginning the aggregation for mcoe
grabbing mcoe from the output object
filling in fuel cost NaNs with rolling averages
Converting the dtypes of: mcoe
aggregate the parts
grabbing ownership_eia860 from the sqlite db
Converting the dtypes of: ownership_eia860
plant
aggregate the parts
grabbing generators_entity_eia from the sqlite db
Converting the dtypes of: generators_entity_eia
plant_gen
aggregate the parts
plant_unit
denormalize plant_unit
grabbing boiler_generator_assn_eia860 from the sqlite db
Converting the dtypes of: boiler_generator_assn_eia860
aggregate the parts
plant_technology
denormalize plant_technology
aggregate the parts
plant_prime_fuel
de

In [9]:
plant_parts_cleaned=  (
    plant_parts_df.
    assign(report_year=lambda x: x.report_date.dt.year,
           plant_id_report_year=lambda x: x.plant_id_pudl.astype(str) 
                              + "_" + x.report_year.astype(str)).
    pipe(cleanstrings_snake, ['record_id_eia']).
    # we'll eventually take this out... once Issue #20
    drop_duplicates(subset=['record_id_eia']).
    set_index('record_id_eia'))

# Test Data Parts

In [10]:
import ferc_eia_connections
test_df_ids = ferc_eia_connections._prep_test_connections(compiler)

# FERC Steam Parts

In [11]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine,freq='AS')

In [12]:
cols_to_use = ['report_year',
'utility_id_ferc1',
'plant_name_ferc1',
'utility_id_pudl', # add
'plant_id_pudl',  #add
'plant_id_ferc1', # pass

'capacity_factor',
'capacity_mw',
'net_generation_mwh',
'opex_fuel',
'opex_fuel_per_mwh',
'fuel_cost',
'fuel_mmbtu',

'construction_year', # add from gens table
'installation_year', # add from the gens table
'primary_fuel_by_mmbtu',
'plant_type', # cooresponding value?
'record_id',]

steam = (pudl_out.plants_steam_ferc1().
         merge(
         pudl_out.fbp_ferc1()[['report_year',
                               'utility_id_ferc1',
                               'plant_name_ferc1',
                               'utility_id_pudl',
                               'fuel_cost',
                               'fuel_mmbtu',
                               'primary_fuel_by_mmbtu',
                              ]],
             on=['report_year',
                 'utility_id_ferc1',
                 'utility_id_pudl',
                 'plant_name_ferc1'
                ],
             how='left')[cols_to_use].
    pipe(pudl.helpers.convert_cols_dtypes, 'ferc1', 'ferc1 plant records').
    dropna())

if 0.9 > len(steam)/len(steam.drop_duplicates(subset=['report_year','utility_id_pudl','plant_id_ferc1'])) < 1.1:
    raise AssertionError('Merge issue with pudl_out.plants_steam_ferc1 and pudl_out.fbp_ferc1.')

steam_cleaned = (
    steam.drop(columns=['utility_id_ferc1','plant_name_ferc1',
                        'plant_id_ferc1','construction_year',
                        'opex_fuel', 'plant_type',]).
    rename(columns={
     'fuel_cost': 'total_fuel_cost',
     'fuel_mmbtu': 'total_mmbtu',
     'opex_fuel_per_mwh': 'fuel_cost_per_mwh',
     'primary_fuel_by_mmbtu': 'fuel_type_code_pudl',
     'record_id': 'record_id_ferc',
        }).
    set_index('record_id_ferc').
    assign(
        fuel_cost_per_mmbtu=lambda x: x.total_fuel_cost/x.total_mmbtu,
        heat_rate_mmbtu_mwh=lambda x: x.total_mmbtu/x.net_generation_mwh,
        plant_id_report_year=lambda x: x.plant_id_pudl.map(str) + "_" + x.report_year.map(str)
))

Converting the dtypes of: ferc1 plant records


# Last minute cleaning

In [13]:
# generate
test_eia = (plant_parts_cleaned.merge(test_df_ids.reset_index().drop(columns=['record_id_ferc'])[['record_id_eia']],
                          left_index=True,
                          right_on=['record_id_eia']).
            drop_duplicates(subset=['record_id_eia']).
            set_index('record_id_eia').
            astype({'total_fuel_cost':float,
                    'total_mmbtu':float}))

In [14]:
test_ferc = (steam_cleaned.merge(test_df_ids.reset_index().drop(columns=['record_id_eia'])[['record_id_ferc']],
                          left_index=True,
                          right_on=['record_id_ferc']).
             drop_duplicates(subset=['record_id_ferc']).
             set_index('record_id_ferc').
             astype({'total_fuel_cost':float,
                     'total_mmbtu':float}))

# Omigosh the actual Matching

In [15]:
import recordlinkage as rl
from recordlinkage.compare import Exact, String, Numeric, Date
from recordlinkage.datasets import load_febrl4

In [16]:
dfA_test = test_ferc
dfB_test = test_eia
dfA = steam_cleaned[steam_cleaned['report_year'] == 2018]
dfB = plant_parts_cleaned[plant_parts_cleaned['report_year'] == 2018]

In [17]:
print(dfA.shape)
print(dfB.shape)

(640, 14)
(92719, 28)


#### Making Comparison Feature Vectors

In [18]:
def make_candidate_links(dfA,dfB, block_col=None):
    indexer = rl.Index()
    indexer.block(block_col)
    return indexer.index(dfA, dfB)

In [19]:
def make_features(dfA, dfB, block_col=None):
    # This cell can take some time to compute.
    compare_cl = rl.Compare(features=[
        Numeric('net_generation_mwh','net_generation_mwh',label='net_generation_mwh',method='exp',scale=1000),
        Numeric('capacity_mw', 'capacity_mw', label='capacity_mw', method='exp'),
        Numeric('total_fuel_cost', 'total_fuel_cost', label='total_fuel_cost', method='exp', offset=2500, scale=1000,missing_value=0.5),
        Numeric('total_mmbtu', 'total_mmbtu', label='total_mmbtu', method='exp', offset=1, scale=100,missing_value=0.5),

        Numeric('capacity_factor', 'capacity_factor', label='capacity_factor'),
        Numeric('fuel_cost_per_mmbtu', 'fuel_cost_per_mmbtu', label='fuel_cost_per_mmbtu'),
        Numeric('fuel_cost_per_mwh', 'fuel_cost_per_mwh', label='fuel_cost_per_mwh'),
        Numeric('heat_rate_mmbtu_mwh', 'heat_rate_mmbtu_mwh', label='heat_rate_mmbtu_mwh'),
        Numeric('heat_rate_mmbtu_mwh', 'heat_rate_mmbtu_mwh', label='heat_rate_mmbtu_mwh'),

        Exact('fuel_type_code_pudl', 'fuel_type_code_pudl', label='fuel_type_code_pudl'),
        Exact('installation_year', 'installation_year', label='installation_year'),
        Exact('utility_id_pudl', 'utility_id_pudl', label='utility_id_pudl'),
        Exact('plant_id_pudl', 'plant_id_pudl', label='plant_id_pudl'),
    ])

    features = compare_cl.compute(make_candidate_links(dfA, dfB, block_col), dfA, dfB)
    return features

In [20]:
features_all = make_features(dfA, dfB, block_col='plant_id_report_year')
features_test = make_features(dfA_test, dfB_test, block_col='plant_id_report_year')

In [21]:
features_all.describe().T

,count,mean,std,min,25%,50%,75%,max
net_generation_mwh,16805.0,0.072636,0.240101,0.0,0.000000e+00,5.496476e-285,1.962402e-47,1.000000
capacity_mw,16805.0,0.090917,0.276447,0.0,3.709913e-181,3.477628e-82,1.011596e-19,1.000000
total_fuel_cost,16805.0,0.001771,0.035613,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
total_mmbtu,16805.0,0.022198,0.131121,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
capacity_factor,16805.0,0.907209,0.124374,0.0,8.837917e-01,9.635876e-01,9.909931e-01,1.000000
fuel_cost_per_mmbtu,16805.0,0.413486,0.453447,0.0,0.000000e+00,0.000000e+00,9.337477e-01,0.999960
fuel_cost_per_mwh,16805.0,0.139109,0.280425,0.0,0.000000e+00,0.000000e+00,1.849496e-02,0.999152
heat_rate_mmbtu_mwh,16805.0,0.358637,0.425193,0.0,0.000000e+00,0.000000e+00,8.466915e-01,0.999999
heat_rate_mmbtu_mwh,16805.0,0.358637,0.425193,0.0,0.000000e+00,0.000000e+00,8.466915e-01,0.999999
fuel_type_code_pudl,16805.0,0.808390,0.393579,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000


In [22]:
features_test.describe().T

,count,mean,std,min,25%,50%,75%,max
net_generation_mwh,198.0,0.179946,0.355716,0.000000,0.000000e+00,2.182818e-73,0.007016,1.000000
capacity_mw,198.0,0.245482,0.409022,0.000000,1.199741e-130,4.126022e-32,0.757858,1.000000
total_fuel_cost,198.0,0.002214,0.031158,0.000000,0.000000e+00,0.000000e+00,0.000000,0.438434
total_mmbtu,198.0,0.005745,0.066020,0.000000,0.000000e+00,0.000000e+00,0.000000,0.899414
capacity_factor,198.0,0.934087,0.103369,0.576673,9.206345e-01,9.865912e-01,0.996129,1.000000
fuel_cost_per_mmbtu,198.0,0.499747,0.465935,0.000000,0.000000e+00,8.450481e-01,0.955291,0.998389
fuel_cost_per_mwh,198.0,0.220457,0.346068,0.000000,0.000000e+00,0.000000e+00,0.399088,0.990634
heat_rate_mmbtu_mwh,198.0,0.433348,0.438321,0.000000,0.000000e+00,4.595451e-01,0.883538,0.999891
heat_rate_mmbtu_mwh,198.0,0.433348,0.438321,0.000000,0.000000e+00,4.595451e-01,0.883538,0.999891
fuel_type_code_pudl,198.0,0.823232,0.382439,0.000000,1.000000e+00,1.000000e+00,1.000000,1.000000


### Classificaiton Models

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
cs= [1, 10, 100, 1000]
cws = ['balanced', None]

ps = {'newton-cg': ['l2', 'none'],
      'lbfgs': ['l2', 'none'],
      'liblinear': ['l1', 'l2'],
      'sag': ['l2', 'none'],
      'saga': ['l1', 'l2', 'elasticnet', 'none'],
     }

results_options = pd.DataFrame()

for solver in solvers:
    for c in cs:
        for cw in cws:
            for p in ps[solver]:
                if p == 'elasticnet':
                    l1_ratios = [.1,.3,.5,.7,.9]
                else:
                    l1_ratios = [None]
                for l1 in l1_ratios:
                    logger.debug(f'train: {solver}: c-{c}, cw-{cw}, p-{p}, l1-{l1}')
                    lrc = rl.LogisticRegressionClassifier(solver=solver, C=c, class_weight=cw, penalty=p, l1_ratio=l1)
                    predict_test = lrc.fit_predict(features_test,match_index=test_df_ids.index)
                    predict = lrc.predict(features_all)
                    results_options = results_options.append(pd.DataFrame(
                        data={'solver':[solver],
                              'precision':[rl.precision(test_df_ids.index, predict_test)],
                              'f_score':[rl.fscore(test_df_ids.index, predict_test)],
                              'c': [c],
                              'cw': [cw],
                              'penalty': [p],
                              'predictions':[len(predict)],
                              'df':[predict],
                              'coef': [lrc.coefficients],
                              'interc': [lrc.intercept],
                                         },
                                   ))


In [25]:
results_options.describe()

,precision,f_score,c,predictions,interc
count,128.000000,128.000000,128.000000,128.000000,128.000000
mean,0.856448,0.567949,277.750000,5406.398438,-5.670608
std,0.006299,0.009255,420.429701,348.976238,1.670446
min,0.830508,0.539394,1.000000,4584.000000,-11.190373
25%,0.852174,0.568915,7.750000,5218.000000,-5.703098
50%,0.854075,0.568915,55.000000,5526.000000,-5.355578
75%,0.858407,0.571429,325.000000,5649.000000,-5.091023
max,0.873786,0.579710,1000.000000,6103.000000,-1.482600


In [26]:
results_options.sample(2)

,solver,precision,f_score,c,cw,penalty,predictions,df,coef,interc
0,saga,0.853448,0.575581,100,None,elasticnet,5631,"MultiIndex([( 'f1_steam_2018_12_1_0_3', ...),...","[2.474044292183461, 2.8612222783554113, 0.0, 0...",-5.206877
0,saga,0.853448,0.575581,100,None,l2,5621,"MultiIndex([( 'f1_steam_2018_12_1_0_3', ...),...","[2.468347645023465, 2.8529713256229217, 0.0509...",-5.227662


In [27]:
pd.DataFrame(index=predict).tail(30)

Empty DataFrame
Columns: []
Index: [(f1_steam_2018_12_432_0_1, 460_dfo_2018_plant_prime_fuel_total_56146), (f1_steam_2018_12_432_0_1, 460_ng_2018_plant_prime_fuel_total_56146), (f1_steam_2018_12_432_0_1, 460_ic_2018_plant_prime_mover_total_56146), (f1_steam_2018_12_432_0_1, 460_st_2018_plant_prime_mover_total_56146), (f1_steam_2018_12_432_0_3, 6516_2018_plant_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic1_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic1a_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic2_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic3_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic4_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic5_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_3, 6516_petroleum_liquids_2018_plant_technology_total_56146), (f1_steam_2018_12_432_0_3, 6516_dfo_2018_plant_prime_fuel_total_56146), (f1_steam_2018_12_432_0_3, 6516_ic_2018_plant_prime_mover_total_56146), (f1_steam_2018_12_432_0_4, 7995_2018_plant_total_56146), (f1_steam_2018_12_432_0_4, 7995_ic1_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_4, 7995_ic2_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_4, 7995_ic3_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_4, 7995_ic4_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_4, 7995_petroleum_liquids_2018_plant_technology_total_56146), (f1_steam_2018_12_432_0_4, 7995_dfo_2018_plant_prime_fuel_total_56146), (f1_steam_2018_12_432_0_4, 7995_ic_2018_plant_prime_mover_total_56146), (f1_steam_2018_12_432_0_5, 7995_2018_plant_total_56146), (f1_steam_2018_12_432_0_5, 7995_ic1_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_5, 7995_ic2_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_5, 7995_ic3_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_5, 7995_ic4_2018_plant_gen_total_56146), (f1_steam_2018_12_432_0_5, 7995_petroleum_liquids_2018_plant_technology_total_56146), (f1_steam_2018_12_432_0_5, 7995_dfo_2018_plant_prime_fuel_total_56146), (f1_steam_2018_12_432_0_5, 7995_ic_2018_plant_prime_mover_total_56146)]

In [28]:
features_all.loc['f1_steam_2018_12_432_0_3']

,net_generation_mwh,capacity_mw,total_fuel_cost,total_mmbtu,capacity_factor,fuel_cost_per_mmbtu,fuel_cost_per_mwh,heat_rate_mmbtu_mwh,heat_rate_mmbtu_mwh,fuel_type_code_pudl,installation_year,utility_id_pudl,plant_id_pudl
record_id_eia,,,,,,,,,,,,,
6190_2018_plant_owned_3265,8.600934e-224,4.148381e-48,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6190_2018_plant_total_3265,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.777822,0.0,0.0,0.0,0.0,0,0,0,1
6190_2018_plant_owned_26253,1.874734e-149,2.602271e-31,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6190_2018_plant_total_26253,0.000000e+00,1.085331e-165,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6190_2018_plant_owned_50111,0.000000e+00,1.054220e-81,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6190_2018_plant_total_50111,0.000000e+00,1.085331e-165,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6516_2018_plant_total_56146,8.906929e-01,2.500000e-01,1.972128e-09,0.000003,0.999047,0.0,0.0,0.0,0.0,1,1,1,1
6190_2_2018_plant_gen_owned_3265,8.600934e-224,4.148381e-48,0.000000e+00,0.000000,0.746446,0.0,0.0,0.0,0.0,0,0,0,1
6190_1_2018_plant_gen_total_3265,2.054551e-72,7.969607e-132,0.000000e+00,0.000000,0.968558,0.0,0.0,0.0,0.0,0,0,0,1


### Test record linkage...

In [29]:
import recordlinkage as rl
from recordlinkage.datasets import load_krebsregister

krebs_X, krebs_true_links = load_krebsregister(missing_values=0)

In [30]:
krebs_X

,,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
id1,id2,,,,,,,,,
22161,38467,1.000000,0.0,0.142857,0.0,1,0.0,1.0,0.0,0.0
38713,75352,0.000000,0.0,0.571429,0.0,1,0.0,0.0,0.0,0.0
13699,32825,0.166667,0.0,0.000000,0.0,0,1.0,1.0,1.0,0.0
22709,37682,0.285714,0.0,1.000000,0.0,1,0.0,0.0,0.0,0.0
2342,69060,0.250000,0.0,0.125000,0.0,1,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52124,53629,1.000000,0.0,0.285714,0.0,1,0.0,0.0,1.0,0.0
30007,76846,0.750000,0.0,0.000000,0.0,1,1.0,0.0,0.0,0.0
50546,59461,0.750000,0.0,0.000000,0.0,1,0.0,1.0,0.0,0.0


In [31]:
krebs_true_links

MultiIndex([(89874, 89876),
            (79126, 84983),
            (40350, 83715),
            (75394, 92002),
            (23323, 27823),
            (31059, 72216),
            (28464, 69899),
            (33613, 64971),
            (23546, 27978),
            (29922, 46075),
            ...
            (62697, 62705),
            (82751, 82753),
            (45892, 59324),
            (33619, 40411),
            (28562, 43893),
            (26162, 27408),
            (71973, 71976),
            (23925, 29112),
            (57711, 99655),
            (18795, 41061)],
           names=['id1', 'id2'], length=20931)

In [32]:
lrc_test = rl.LogisticRegressionClassifier()

In [33]:
predict = lrc_test.fit_predict(krebs_X, krebs_true_links)

In [34]:
predict

MultiIndex([(89874, 89876),
            (79126, 84983),
            (40350, 83715),
            (75394, 92002),
            (23323, 27823),
            (31059, 72216),
            (28464, 69899),
            (33613, 64971),
            (23546, 27978),
            (29922, 46075),
            ...
            (62697, 62705),
            (82751, 82753),
            (45892, 59324),
            (33619, 40411),
            (28562, 43893),
            (26162, 27408),
            (71973, 71976),
            (23925, 29112),
            (57711, 99655),
            (18795, 41061)],
           names=['id1', 'id2'], length=20885)